# SciPy.org's [Numpy](http://www.numpy.org/)

Numpy provides a high-performance multidimensional array object.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Why-numpy?" data-toc-modified-id="Why-numpy?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Why numpy?</a></span></li><li><span><a href="#Creating-(simple)-arrays-in-Numpy" data-toc-modified-id="Creating-(simple)-arrays-in-Numpy-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating (simple) <a href="https://docs.scipy.org/doc/numpy-dev/user/quickstart.html" target="_blank">arrays</a> in Numpy</a></span></li><li><span><a href="#1D-arrays" data-toc-modified-id="1D-arrays-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>1D arrays</a></span></li><li><span><a href="#2D-arrays" data-toc-modified-id="2D-arrays-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>2D arrays</a></span></li><li><span><a href="#N-dimensional-arrays" data-toc-modified-id="N-dimensional-arrays-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>N-dimensional arrays</a></span></li><li><span><a href="#Slicing" data-toc-modified-id="Slicing-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Slicing</a></span></li><li><span><a href="#Boolean-array-indexing" data-toc-modified-id="Boolean-array-indexing-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Boolean array indexing</a></span></li><li><span><a href="#Elementwise-(vectorial-vectorial-and-vectorial-scalar)-math" data-toc-modified-id="Elementwise-(vectorial-vectorial-and-vectorial-scalar)-math-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Elementwise (vectorial-vectorial and vectorial-scalar) math</a></span></li><li><span><a href="#Matricial-math" data-toc-modified-id="Matricial-math-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Matricial math</a></span></li><li><span><a href="#Broadcasting" data-toc-modified-id="Broadcasting-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Broadcasting</a></span></li><li><span><a href="#How-fast-is-Numpy's-array-math?" data-toc-modified-id="How-fast-is-Numpy's-array-math?-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>How fast is Numpy's array math?</a></span></li><li><span><a href="#Structured-arrays" data-toc-modified-id="Structured-arrays-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Structured arrays</a></span></li><li><span><a href="#Output-data-to-an-ASCII-file" data-toc-modified-id="Output-data-to-an-ASCII-file-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Output data to an ASCII file</a></span></li></ul></div>

## Why numpy?

Good running times.

In [2]:
try:
    import numpy as np
except:
    !pip3 install numpy
    import numpy as np

* Lets define a list and compute the sum of its elements, timing it:

In [2]:
l = list(range(0,100000)); print(type(l), l[:10])
%timeit sum(l)

<class 'list'> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
1.73 ms ± 29.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


* An now, lets create a numpy's array and time the sum of its elements:

In [194]:
A = np.arange(0, 100000); print(type(a), a[:10])
%timeit np.sum(A)

<class 'numpy.ndarray'> [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
103 µs ± 7.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


* And what about a *pure* C implementation of an equivalent computation: 

In [195]:
!cat sum_array.c
!gcc -O3 sum_array.c -o sum_array
%timeit !./sum_array

#include <stdio.h>
#include <time.h>
#include "sum_array_lib.c"

#define N 100000

int main() {
  double a[N];
  int i;
  clock_t start, end;
  double cpu_time;
  for(i=0; i<N; i++) {
    a[i] = i;
  }
  start = clock();
  double sum = sum_array(a,N);
  end = clock();
  printf("%f ", sum);
  cpu_time = ((double) (end - start)) / CLOCKS_PER_SEC;
  cpu_time *= 1000000;
  printf("%f usegs\n", cpu_time);
}
4999950000.000000 166.000000 usegs
4999950000.000000 166.000000 usegs
4999950000.000000 151.000000 usegs
4999950000.000000 150.000000 usegs
4999950000.000000 150.000000 usegs
4999950000.000000 165.000000 usegs
4999950000.000000 167.000000 usegs
4999950000.000000 150.000000 usegs
4999950000.000000 173.000000 usegs
4999950000.000000 151.000000 usegs
4999950000.000000 167.000000 usegs
4999950000.000000 151.000000 usegs
4999950000.000000 150.000000 usegs
4999950000.000000 151.000000 usegs
4999950000.000000 150.000000 usegs
4999950000.000000 150.000000 usegs
4999950000.000000 150.000000 usegs

* Another example:

In [85]:
# Example extracted from https://github.com/pyHPC/pyhpc-tutorial
lst = range(1000000)

for i in lst[:10]:
    print(i, end=' ')
print()

%timeit [i + 1 for i in lst] # A Python list comprehension (iteration happens in C but with PyObjects)
x = [i + 1 for i in lst]

print(x[:10])

arr = np.arange(1000000) # A NumPy list of integers
%timeit arr + 1 # Use operator overloading for nice syntax, now iteration is in C with ints
y = arr + 1

print(y[:10])

0 1 2 3 4 5 6 7 8 9 
207 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
5.17 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
[ 1  2  3  4  5  6  7  8  9 10]


* Looking for information of numpy's *something*:

In [86]:
np.lookfor('invert')

Search results for 'invert'
---------------------------
numpy.bitwise_not
    Compute bit-wise inversion, or bit-wise NOT, element-wise.
numpy.matrix.getI
    Returns the (multiplicative) inverse of invertible `self`.
numpy.in1d
    Test whether each element of a 1-D array is also present in a second array.
numpy.isin
    Calculates `element in test_elements`, broadcasting over `element` only.
numpy.transpose
    Permute the dimensions of an array.
numpy.linalg.inv
    Compute the (multiplicative) inverse of a matrix.
numpy.linalg.pinv
    Compute the (Moore-Penrose) pseudo-inverse of a matrix.
numpy.random.SFC64
    BitGenerator for Chris Doty-Humphrey's Small Fast Chaotic PRNG.
numpy.linalg.tensorinv
    Compute the 'inverse' of an N-dimensional array.
numpy.linalg.matrix_power
    Raise a square matrix to the (integer) power `n`.

* Remember that it's possible to use the tabulator to extend some command or to use a wildcard in Ipython to get the numpy's stuff:

In [87]:
np.*?

## Creating (simple) [arrays](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html) in Numpy
A simple array is a grid of values, all of the same type, indexed by a tuple of nonnegative integers.

## 1D arrays

* Creating an empty array:

In [196]:
A = np.array([], dtype=np.uint8)
A

array([], dtype=uint8)

* Creating an array using a list:

In [197]:
A = np.array([1, 2, 3])
print(type([1, 2, 3]))
print(type(A))

<class 'list'>
<class 'numpy.ndarray'>


* Getting the number of dimensions of an array:

In [198]:
print(A.ndim)

1


* Printing an array:

In [199]:
print(A)

[1 2 3]


* Printing the *shape* (which always is a tuple) of an array:

In [200]:
print(A.shape)

(3,)


* Native Python's [`len()`](https://docs.python.org/3.6/library/functions.html#len) also works:

In [201]:
print(len(A))

3


* A more exotic definition using [`linspace()`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linspace.html):

In [94]:
np.linspace(1., 4., 6)

array([1. , 1.6, 2.2, 2.8, 3.4, 4. ])

* Arrays can be created from different types of contaniers (which store complex numbers in this case):

In [202]:
C = [[1,1.0],(1+1j,.3)]
print(type(C), type(C[0]), type(C[1]))
X = np.array(C)
X

<class 'list'> <class 'list'> <class 'tuple'>


array([[1. +0.j, 1. +0.j],
       [1. +1.j, 0.3+0.j]])

* Accessing to an element:

In [203]:
print(A, A[0], A[1])

[1 2 3] 1 2


In [204]:
A[0] = 0
print(A)

[0 2 3]


* Appending elements:

In [205]:
A = np.append(A, 4)
A

array([0, 2, 3, 4])

## 2D arrays

* Creating a 2D array with two 1D arrays:

In [4]:
B = np.array([[1,2,3],[4,5,6]])
print(B)
print(B.shape)
print(B[1, 2]) # [row, column]
print(B[0, 1])

[[1 2 3]
 [4 5 6]]
(2, 3)
6
2


* With zeroes:

In [207]:
A = np.zeros((5,5))
print(A)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


* The default dtype is `float64`:

In [208]:
print(type(A[0][0]))

<class 'numpy.float64'>


* With ones:

In [209]:
A = np.ones((5,5))
print(A)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


* With an arbitrary scalar:

In [210]:
A = np.full((5,5), 2)
print(A)

[[2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]]


* The identity matrix:

In [211]:
A = np.eye(5)
print(A)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


* With random data:

In [212]:
A = np.random.random((5,5))
print(A)

[[0.09237174 0.27764355 0.8858637  0.96899807 0.21275904]
 [0.82366172 0.18838078 0.19071355 0.24948483 0.16675471]
 [0.2263183  0.87321448 0.60758063 0.60334629 0.10978271]
 [0.94691356 0.47788906 0.43900904 0.3870766  0.53429403]
 [0.36243192 0.28660014 0.95716892 0.05685688 0.66822065]]


In [213]:
# Always random
A = np.random.random((5,5))
print(A)

[[0.55192536 0.13365655 0.99482323 0.01777324 0.00568849]
 [0.71230274 0.37978767 0.7575434  0.8872667  0.8936947 ]
 [0.39790792 0.09040365 0.05923547 0.78224393 0.3668818 ]
 [0.95190205 0.10107778 0.97884516 0.65764402 0.7347871 ]
 [0.48975339 0.19293206 0.26147661 0.15111748 0.06126388]]


* Filled with [arbitrary](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.empty_like.html) data and with a previously defined shape:

In [214]:
B = np.empty_like(A) # The content could be any
print(B)

[[0.55192536 0.13365655 0.99482323 0.01777324 0.00568849]
 [0.71230274 0.37978767 0.7575434  0.8872667  0.8936947 ]
 [0.39790792 0.09040365 0.05923547 0.78224393 0.3668818 ]
 [0.95190205 0.10107778 0.97884516 0.65764402 0.7347871 ]
 [0.48975339 0.19293206 0.26147661 0.15111748 0.06126388]]


* With a 1D list comprehension:

In [215]:
A = np.array([i for i in range(5)])
print(A, A[1], A.shape)

[0 1 2 3 4] 1 (5,)


* With a 2D list comprehension:

In [41]:
A = np.array([[j+i*4 for j in range(4)] for i in range(5)])
print(A, A.shape)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]] (5, 4)


* Accessing to a row of a matrix:

In [42]:
A[1] # Get row 2

array([4, 5, 6, 7])

* Accessing to an element of a matrix:

In [43]:
A[1][2] # [row][column]

6

In [44]:
A[1,2] # [row, column]

6

* Be careful:

In [45]:
timeit A[1][2]

654 ns ± 30.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [46]:
timeit A[1,2] # This is faster than a[1][2]

328 ns ± 20.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


* Getting elements of a matrix using "integer array indexing":

In [47]:
print(A)
print(A[[0, 1, 2], [3, 2, 1]])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]]
[3 6 9]


* The same integer array indexing using comprehension lists:

In [48]:
print(A[np.array([i for i in range(3)]), np.array([i for i in range(3,0,-1)])])

[3 6 9]


* The same using [`np.arange()`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.arange.html):

In [49]:
print(np.arange(3))
print(np.arange(3,0,-1))
print(A[np.arange(3), np.arange(3,0,-1)])

[0 1 2]
[3 2 1]
[3 6 9]


* Reshaping:

In [50]:
A.shape

(5, 4)

In [51]:
np.reshape(A, (10, 2))

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11],
       [12, 13],
       [14, 15],
       [16, 17],
       [18, 19]])

In [71]:
B = np.reshape(A, (10, 2), order='C') # C -> C language (the default behaviour)
B

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11],
       [12, 13],
       [14, 15],
       [16, 17],
       [18, 19]])

In [72]:
np.isfortran(B)

False

In [64]:
# As you can see, by default, numpy runs the 2D arrays by rows when
# the source matrix A is read and the destination matrix B is "written":
# A[0, 0] == B[0, 0]
# A[0, 1] == B[0, 1]
# A[0, 2] == B[1, 0]
# A[0, 3] == B[1, 1]
# :
A

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [65]:
print(A[0, 3], B[1, 1])

3 3


In [73]:
B = np.reshape(A, (10, 2), order='F') # F -> Fortran language
B

array([[ 0,  2],
       [ 4,  6],
       [ 8, 10],
       [12, 14],
       [16, 18],
       [ 1,  3],
       [ 5,  7],
       [ 9, 11],
       [13, 15],
       [17, 19]])

In [74]:
# Using the Fortran ordering, numpy runs the 2D array by columns:
# A[0, 0] == B[0, 0]
# A[1, 0] == B[1, 0]
# A[2, 0] == B[2, 0]
# A[3, 0] == B[3, 0]
# A[4, 0] == B[4, 0]
# A[0, 1] == B[5, 0]
# :
A

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [75]:
print(A[0, 1], B[5, 0])

1 1


In [76]:
np.isfortran(B)

True

* Views and copies:

In [82]:
# https://stackoverflow.com/questions/56090021/list-comprehension-python-prime-numbers
Primes_less_than_100 = np.array([x for x in range(2,100) if not any([x % y == 0 for y in range(2, int(x/2)+1)])])
Primes_less_than_100

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [83]:
A = Primes_less_than_100 # This is a copy of pointers
A

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [84]:
A[0]=1
A

array([ 1,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [85]:
Primes_less_than_100

array([ 1,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [88]:
id(A)

4710565328

In [89]:
id(Primes_less_than_100)

4710565328

In [99]:
Primes_less_than_100 = np.array([x for x in range(2,100) if not any([x % y == 0 for y in range(2, int(x/2)+1)])])
A = np.copy(Primes_less_than_100)

In [100]:
id(A)

4589318192

In [101]:
id(Primes_less_than_100)

4720969888

In [102]:
Primes_less_than_100[0]

2

In [103]:
A[0] = 1

In [104]:
print(Primes_less_than_100[0], A[0])

2 1


In [98]:
timeit A=Primes_less_than_100 # This is much faster, depending on the size of the array

55.5 ns ± 4.35 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [105]:
%timeit A = np.copy(Primes_less_than_100)

4.53 µs ± 211 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## N-dimensional arrays

In [226]:
A = np.ndarray((2,3,4,2))
A

array([[[[-2.68156159e+154, -2.68156159e+154],
         [ 1.23516411e-322,  0.00000000e+000],
         [ 2.35541533e-312,  5.02034658e+175],
         [ 3.54132133e-033,  2.19210650e-056]],

        [[ 1.22203826e+161,  2.42227138e-052],
         [ 1.47763641e+248,  1.16096346e-028],
         [ 7.69165785e+218,  1.35617292e+248],
         [ 6.50450970e-038,  1.21497568e-046]],

        [[ 5.98250116e+174,  2.21368728e+160],
         [ 4.26232219e-096,  6.32299154e+233],
         [ 6.48224638e+170,  5.22411352e+257],
         [ 5.74020278e+180,  8.37174974e-144]]],


       [[[ 1.41529402e+161,  6.00736899e-067],
         [ 3.46027944e+097,  6.23815118e-038],
         [ 3.58769076e+126,  2.22809574e-312],
         [-2.68156159e+154, -2.68156159e+154]],

        [[ 2.37663529e-312,  2.14321575e-312],
         [ 2.44029516e-312,  2.22809558e-312],
         [ 6.79038654e-313,  6.79038653e-313],
         [ 2.48273508e-312,  6.79038654e-313]],

        [[ 2.22809558e-312,  2.20687562e-312],
 

In [227]:
A.shape

(2, 3, 4, 2)

In [228]:
# The same can be done with:
A = np.ndarray(2*3*4*2).reshape(2, 3, 4, 2)
A.shape

(2, 3, 4, 2)

## Slicing

In [229]:
A = np.array([[j+i*5 for j in range(10)] for i in range(5)])
A

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

* Get all rows of a matrix (the whole matrix):

In [230]:
A[:]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [231]:
A[:,:]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [232]:
timeit A[:]

492 ns ± 33.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [233]:
timeit A[:,:] # This is slightly slower than 'a[:]'

626 ns ± 61.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [234]:
# Notation: [starting index : stoping index : step]
# By default, start = 0, stop = maximum, step = 1
A[::]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [235]:
timeit A[::] # Identical to 'a[:]'

472 ns ± 20 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [236]:
timeit A # But not to 'a'

43.2 ns ± 2.54 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [237]:
A[0:]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [238]:
A[0::]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [239]:
A[:A.shape[1]]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [240]:
A[:A.shape[1]:]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [241]:
A[:A.shape[1]:1]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

* Get all rows of a matrix, except the first one:

In [242]:
A[1:]

array([[ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [243]:
A[1::]

array([[ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

* Get the first two rows of a matrix:

In [244]:
A[0:2]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]])

* Get the even rows of a matrix:

In [245]:
A[0::2]

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

* Get the odd rows of a matrix:

In [246]:
A[1::2]

array([[ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19, 20, 21, 22, 23, 24]])

* Get the odd columns of a matrix:

In [247]:
A[:,1::2]

array([[ 1,  3,  5,  7,  9],
       [ 6,  8, 10, 12, 14],
       [11, 13, 15, 17, 19],
       [16, 18, 20, 22, 24],
       [21, 23, 25, 27, 29]])

* Getting the second row:

In [248]:
A[1,:]

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

* Getting the third column:

In [249]:
A[:,2]

array([ 2,  7, 12, 17, 22])

* Getting a top-left $2\times 2$ submatrix:

In [250]:
A[:2,:2]

array([[0, 1],
       [5, 6]])

* Getting a bottom-right $2\times 2$ submatrix:

In [251]:
A[A.shape[0]-2:,A.shape[1]-2:]

array([[23, 24],
       [28, 29]])

 ## Boolean array indexing

* Finding the elements bigger than ...

In [252]:
bool_idx = (A>12)
print(bool_idx)

[[False False False False False False False False False False]
 [False False False False False False False False  True  True]
 [False False False  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]]


* Printing the elements bigger than ...

In [253]:
print(A[bool_idx])

[13 14 13 14 15 16 17 18 19 15 16 17 18 19 20 21 22 23 24 20 21 22 23 24
 25 26 27 28 29]


## Elementwise (vectorial-vectorial and vectorial-scalar) math

* Create an zero-ed matrix:

In [265]:
A = np.zeros((5,5), np.int32)
print(A)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


* Change to 1 from coordinate (1,1) to coordinate (4,4):

In [266]:
A[1:4,1:4] = 1
print(A)

[[0 0 0 0 0]
 [0 1 1 1 0]
 [0 1 1 1 0]
 [0 1 1 1 0]
 [0 0 0 0 0]]


* Vectorial-scalar addition:

In [267]:
A[1:4, 1:4] += 1
print(A)

[[0 0 0 0 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 0 0 0 0]]


* A new matrix:

In [268]:
B = np.ones((5,5), np.int32)
print(B)

[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]


* Vectorial addition:

In [269]:
C = A + B
print(C)

[[1 1 1 1 1]
 [1 3 3 3 1]
 [1 3 3 3 1]
 [1 3 3 3 1]
 [1 1 1 1 1]]


* Vectorial substraction:

In [271]:
D = C - B
print(D)

[[0 0 0 0 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 0 0 0 0]]


* Vectorial multiplication (not matrix multiplication!):

In [274]:
C = C * D
print(C)

[[ 0  0  0  0  0]
 [ 0 24 24 24  0]
 [ 0 24 24 24  0]
 [ 0 24 24 24  0]
 [ 0  0  0  0  0]]


* Floating-point vectorial division:

In [275]:
C = C / B
print(C)

[[ 0.  0.  0.  0.  0.]
 [ 0. 24. 24. 24.  0.]
 [ 0. 24. 24. 24.  0.]
 [ 0. 24. 24. 24.  0.]
 [ 0.  0.  0.  0.  0.]]


* Fixed-point (integer) vectorial division:

In [276]:
C = D // B
print(C)

[[0 0 0 0 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 2 2 2 0]
 [0 0 0 0 0]]


## Matricial math
Basic matrix computations.

* Let's define a "chessboard" matrix:

In [166]:
A = np.array([[(i+j)%2 for j in range(10)] for i in range(10)])
print(A, A.shape)

[[0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]] (10, 10)


... and a 1-column matrix:

In [167]:
B = np.array([[1] for i in range(10)])
print(B, B.shape)

[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]] (10, 1)


* Product matrix-matrix:

In [169]:
C = A @ B
print(C)

[[5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]]


* Sum of all elements of a matrix:

In [170]:
print(np.sum(C))

50


In [171]:
print(np.sum(A))

50


* Compute the maximum of a matrix:

In [172]:
print(np.max(C))

5


* Matrix transpose:

In [173]:
print(C.T, C.T.shape, C.shape)

[[5 5 5 5 5 5 5 5 5 5]] (1, 10) (10, 1)


* Determinant:

In [184]:
np.linalg.det(A)

0.0

* Inverse:

In [281]:
R = np.random.rand(5,5)
iR = np.linalg.inv(R)
print(iR)

[[-5.12888137  1.73199653 -6.58839234  3.05296518  4.85363349]
 [-2.78890167  2.46096242 -3.9964955   3.64685536  0.50914113]
 [-1.36149291  0.22101343 -2.15299491  3.70117595 -0.27580294]
 [ 1.62181246 -0.85699328  4.08305935 -3.26864222 -0.40910973]
 [ 8.33361978 -3.50016404  9.1192575  -6.6854481  -4.4056587 ]]


In [282]:
np.round(R @ iR)

array([[ 1.,  0., -0.,  0.,  0.],
       [-0.,  1., -0.,  0.,  0.],
       [ 0., -0.,  1., -0.,  0.],
       [-0., -0., -0.,  1.,  0.],
       [-0., -0., -0., -0.,  1.]])

In [283]:
R @ iR

array([[ 1.00000000e+00,  4.44089210e-16, -8.88178420e-16,
         8.88178420e-16,  1.11022302e-16],
       [-4.44089210e-16,  1.00000000e+00, -8.88178420e-16,
         0.00000000e+00,  1.66533454e-16],
       [ 0.00000000e+00, -1.11022302e-16,  1.00000000e+00,
        -2.22044605e-16,  8.32667268e-17],
       [-4.44089210e-16, -2.22044605e-16, -8.88178420e-16,
         1.00000000e+00,  0.00000000e+00],
       [-1.77635684e-15, -4.44089210e-16, -8.88178420e-16,
        -8.88178420e-16,  1.00000000e+00]])

In [183]:
np.round(iR @ R)

array([[ 1., -0.,  0., -0., -0.],
       [-0.,  1.,  0.,  0., -0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [284]:
iR @ R

array([[ 1.00000000e+00, -4.44089210e-16, -8.88178420e-16,
        -2.22044605e-15, -4.44089210e-16],
       [-2.22044605e-16,  1.00000000e+00, -5.55111512e-17,
        -7.77156117e-16, -4.44089210e-16],
       [-3.05311332e-16, -1.66533454e-16,  1.00000000e+00,
        -7.49400542e-16, -2.22044605e-16],
       [ 2.77555756e-16, -1.11022302e-16,  2.77555756e-16,
         1.00000000e+00,  0.00000000e+00],
       [ 8.88178420e-16,  0.00000000e+00,  4.44089210e-16,
         1.33226763e-15,  1.00000000e+00]])

* Pseudo-inverse:

In [277]:
R = np.random.rand(5,4)
iR = np.linalg.pinv(R)
print(iR)

[[ 0.87334274  0.9288675  -0.44293668 -0.84529282 -0.1961291 ]
 [ 1.19208836 -1.74684064  1.07154311  0.88392885 -1.39298442]
 [-1.12502303  1.629837    0.03312958 -1.16543196  2.0888105 ]
 [-0.54659534 -0.677702   -0.05585232  1.91545933 -0.29331457]]


In [278]:
np.round(R @ iR)

array([[ 1.,  0.,  0., -0., -0.],
       [ 0.,  1., -0.,  0.,  0.],
       [ 0., -0.,  1.,  0.,  0.],
       [-0.,  0.,  0.,  1., -0.],
       [-0.,  0.,  0., -0.,  1.]])

In [285]:
R @ iR

array([[ 1.00000000e+00,  4.44089210e-16, -8.88178420e-16,
         8.88178420e-16,  1.11022302e-16],
       [-4.44089210e-16,  1.00000000e+00, -8.88178420e-16,
         0.00000000e+00,  1.66533454e-16],
       [ 0.00000000e+00, -1.11022302e-16,  1.00000000e+00,
        -2.22044605e-16,  8.32667268e-17],
       [-4.44089210e-16, -2.22044605e-16, -8.88178420e-16,
         1.00000000e+00,  0.00000000e+00],
       [-1.77635684e-15, -4.44089210e-16, -8.88178420e-16,
        -8.88178420e-16,  1.00000000e+00]])

In [279]:
np.round(iR @ R)

array([[ 1., -0., -0., -0.],
       [ 0.,  1., -0., -0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [286]:
iR @ R

array([[ 1.00000000e+00, -4.44089210e-16, -8.88178420e-16,
        -2.22044605e-15, -4.44089210e-16],
       [-2.22044605e-16,  1.00000000e+00, -5.55111512e-17,
        -7.77156117e-16, -4.44089210e-16],
       [-3.05311332e-16, -1.66533454e-16,  1.00000000e+00,
        -7.49400542e-16, -2.22044605e-16],
       [ 2.77555756e-16, -1.11022302e-16,  2.77555756e-16,
         1.00000000e+00,  0.00000000e+00],
       [ 8.88178420e-16,  0.00000000e+00,  4.44089210e-16,
         1.33226763e-15,  1.00000000e+00]])

## Broadcasting
In vectorized operations, NumPy "extends" scalars and arrays with one of its dimensions equal to 1 to the size of the other(s) array(s).

In [288]:
A = np.ones((5,3))
A

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [289]:
B = np.arange(1)
B

array([0])

In [290]:
B += 1
B

array([1])

* Broadcasting of a $1\times 1$ matrix:

In [291]:
A + B # 'A' is 5x3 and 'B' is 1x1

array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]])

* Broadcasting of a $1\times 3$ matrix:

In [292]:
B = np.arange(3)
B

array([0, 1, 2])

In [293]:
A + B # 'a' is 5x3 and 'b' is '1x3'

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

* Broadcasting of a $5\times 1$ matrix:

In [294]:
B = np.arange(5)
B

array([0, 1, 2, 3, 4])

In [295]:
B = b.reshape((5,1)) # (Rows, Columns)
B

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [296]:
A + B

array([[1., 1., 1.],
       [2., 2., 2.],
       [3., 3., 3.],
       [4., 4., 4.],
       [5., 5., 5.]])

* One of the dimensions must match to broadcast the smaller array. Otherwise a `ValueError: frames are not aligned` is thrown.

In [300]:
B = np.arange(4)[:, None]
B

array([[0],
       [1],
       [2],
       [3]])

In [301]:
A.shape

(5, 3)

In [302]:
B.shape

(4, 1)

In [308]:
try:
    A + B
except ValueError as e:
    print("ValueError exception: ", end='')
    if hasattr(e, 'message'):
        print(e.message)
    else:
        print(e)

ValueError exception: operands could not be broadcast together with shapes (5,3) (4,1) 


## How fast is Numpy's array math?

In [309]:
A = np.array([[(i*10+j) for j in range(10)] for i in range(10)])
print(A, A.shape)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]] (10, 10)


In [310]:
A[:1] # First row (a matrix)

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [311]:
A[:1].shape

(1, 10)

In [312]:
A[:1][0] # First element of a matrix of one elment (a vector)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [314]:
A[:1][0].shape

(10,)

In [315]:
B = A[:1][0]
B

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

* Add `B[]` to all the rows of `A[][]` using scalar arithmetic:

In [316]:
C = np.empty_like(A)
def add():
    for i in range(A.shape[1]):
        for j in range(A.shape[0]):
            C[i, j] = A[i, j] + B[j]
%timeit add()
print(C)

124 µs ± 9.76 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[[  0   2   4   6   8  10  12  14  16  18]
 [ 10  12  14  16  18  20  22  24  26  28]
 [ 20  22  24  26  28  30  32  34  36  38]
 [ 30  32  34  36  38  40  42  44  46  48]
 [ 40  42  44  46  48  50  52  54  56  58]
 [ 50  52  54  56  58  60  62  64  66  68]
 [ 60  62  64  66  68  70  72  74  76  78]
 [ 70  72  74  76  78  80  82  84  86  88]
 [ 80  82  84  86  88  90  92  94  96  98]
 [ 90  92  94  96  98 100 102 104 106 108]]


* Add `B[]` to all the rows of `B[][]` using vectorial computation:

In [317]:
C = np.empty_like(A)
def add():
    for i in range(A.shape[1]):
        C[i, :] = A[i, :] + B
%timeit add()
print(C)

50.9 µs ± 8.46 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[[  0   2   4   6   8  10  12  14  16  18]
 [ 10  12  14  16  18  20  22  24  26  28]
 [ 20  22  24  26  28  30  32  34  36  38]
 [ 30  32  34  36  38  40  42  44  46  48]
 [ 40  42  44  46  48  50  52  54  56  58]
 [ 50  52  54  56  58  60  62  64  66  68]
 [ 60  62  64  66  68  70  72  74  76  78]
 [ 70  72  74  76  78  80  82  84  86  88]
 [ 80  82  84  86  88  90  92  94  96  98]
 [ 90  92  94  96  98 100 102 104 106 108]]


* Add `B[]` to all the rows of `A[][]` using fully vectorial computation:

In [318]:
%timeit C = A + B # <- broadcasting is faster
print(C)

4.23 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
[[  0   2   4   6   8  10  12  14  16  18]
 [ 10  12  14  16  18  20  22  24  26  28]
 [ 20  22  24  26  28  30  32  34  36  38]
 [ 30  32  34  36  38  40  42  44  46  48]
 [ 40  42  44  46  48  50  52  54  56  58]
 [ 50  52  54  56  58  60  62  64  66  68]
 [ 60  62  64  66  68  70  72  74  76  78]
 [ 70  72  74  76  78  80  82  84  86  88]
 [ 80  82  84  86  88  90  92  94  96  98]
 [ 90  92  94  96  98 100 102 104 106 108]]


## Structured arrays

* Create a 1D array of (two) records, where each record has the structure (int, float, char[10]).

In [319]:
X = np.array([(1, 2., 'Hello'), (3, 4., "World")],
             dtype=[('first', 'i4'),('second', 'f4'), ('third', 'S10')])
X

array([(1, 2., b'Hello'), (3, 4., b'World')],
      dtype=[('first', '<i4'), ('second', '<f4'), ('third', 'S10')])

* Get the first element of every record:

In [320]:
X['first']

array([1, 3], dtype=int32)

* Get the first record:

In [321]:
X[0]

(1, 2., b'Hello')

* Get the second element of every record:

In [322]:
X['second']

array([2., 4.], dtype=float32)

* Third element of every record:

In [323]:
X['third']

array([b'Hello', b'World'], dtype='|S10')

## Output data to an ASCII file

In [324]:
Data = np.array([[1., 200.], [2., 150.], [3., 250.]])
np.savetxt('data.txt', Data)
!cat Data.txt

1.000000000000000000e+00 2.000000000000000000e+02
2.000000000000000000e+00 1.500000000000000000e+02
3.000000000000000000e+00 2.500000000000000000e+02


* Input data from an ASCII file:

In [325]:
np.genfromtxt('data.txt')

array([[  1., 200.],
       [  2., 150.],
       [  3., 250.]])

* Output data to a binary file:

In [327]:
ofile = open("data.float64", mode='wb')
Data.tofile(ofile)

* Input data from a binary file:

In [328]:
np.fromfile('data.float64', dtype=np.float64)

array([  1., 200.,   2., 150.,   3., 250.])

* In order to know the lenguaje endianess, let's create a binary file using C:

In [330]:
!cat create_float64.c
!gcc create_float64.c -o create_float64
!./create_float64

#include <stdio.h>

#define N 10

int main() {
  double a[N];
  int i;
  FILE *ofile = fopen("data.float64", "wb");
  for(i=0; i<N; i++) {
    a[i] = i;
  }
  fwrite(a, sizeof(double), N, ofile);
  fclose(ofile);
  fprintf(stderr,"create_float64: done\n");
}
create_float64: done


* Input data from the file created in C:

In [331]:
np.fromfile('data.float64', dtype=np.float64)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])